In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np

In [20]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [15]:
params = list(net.parameters())
print(len(params))
for i in range(10):
    print(params[i].size())

10
torch.Size([6, 1, 3, 3])
torch.Size([6])
torch.Size([16, 6, 3, 3])
torch.Size([16])
torch.Size([120, 576])
torch.Size([120])
torch.Size([84, 120])
torch.Size([84])
torch.Size([10, 84])
torch.Size([10])


In [33]:
from torchsummary import summary
net = Net()
summary(net.cuda(), (1, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 30, 30]              60
            Conv2d-2           [-1, 16, 13, 13]             880
            Linear-3                  [-1, 120]          69,240
            Linear-4                   [-1, 84]          10,164
            Linear-5                   [-1, 10]             850
Total params: 81,194
Trainable params: 81,194
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.06
Params size (MB): 0.31
Estimated Total Size (MB): 0.38
----------------------------------------------------------------


In [55]:
input = torch.randn(1, 1, 32, 32)
out = net(input.to(device))
print(out)

tensor([[-0.1618, -0.0338,  0.1842, -0.0050,  0.0413,  0.0424,  0.0357, -0.0095,
         -0.0160, -0.0237]], device='cuda:0', grad_fn=<AddmmBackward>)


In [34]:
net.zero_grad()
out.backward(torch.randn(1, 10))

## loss functions

In [41]:
torch.cuda.current_device()
torch.cuda.get_device_name(0)
torch.cuda.is_available()

True

In [48]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

Using device: cuda



In [85]:
input = torch.randn(1, 1, 32, 32, requires_grad=True)
output = net(input.to(device))
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target.to(device))
print(loss)

tensor(0.2495, device='cuda:0', grad_fn=<MseLossBackward>)


In [68]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

## backpropagation.

In [77]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.], device='cuda:0')


In [78]:
loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad after backward
tensor([ 0.0137, -0.0096, -0.0007,  0.0014,  0.0173,  0.0037], device='cuda:0')


In [86]:
output.backward(torch.randn(1, 10).to(device))
print(input.grad)

tensor([[[[ 0.0000e+00,  2.6972e-04,  9.4668e-04,  ...,  6.6146e-04,
            0.0000e+00,  0.0000e+00],
          [ 2.8879e-04, -2.7014e-04, -3.5417e-04,  ..., -2.0832e-04,
            0.0000e+00,  0.0000e+00],
          [-4.2964e-04, -5.8947e-04,  1.3116e-04,  ...,  8.1000e-05,
            0.0000e+00,  0.0000e+00],
          ...,
          [-1.2044e-04, -9.4067e-05,  1.1589e-04,  ...,  2.3325e-04,
            0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
            0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
            0.0000e+00,  0.0000e+00]]]])


## update the weights.

In [87]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [90]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input.to(device))
loss = criterion(output, target.to(device))
loss.backward()
optimizer.step()    # Does the update